수집할 데이터  
1) 파일로 존재: pandas --> read_XXX, 여러개 파일 os/sys/glob 일괄처리  
2) 정보를 규칙에 맞게 주고받은 openAPI를 이용하는 경우 --> http통신(utllib, requests, etc)
+ json, xml: 패키지로 json, bs4
3) 내가 수집할 정보가 사이트상 --> 크롤링 정적인 주소

In [71]:
# 필요한 패키지
import requests # urllib, request, etc
# 데이터 처리
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# 기타 필요 : 정규식: 시낙
import re
import time

In [9]:
# 여청할 주소를 full 완성을 해야함
# 브라우저의 네트워크 속성에서 숨겨져있는 주소들을 파악
# selectDate = YYYY.MM.DD
# outrentPage = #N

date = '2017.03.28'
page = '1'             # 어차피 주소를 문자열로 만들것이니 숫자여도 문자로 세팅한다
url = f'https://dart.fss.or.kr/dsac001/mainAll.do?selectDate={date}&sort=&series=&mdayCnt=0&currentPage={page}'
print(url)

https://dart.fss.or.kr/dsac001/mainAll.do?selectDate=2017.03.28&sort=&series=&mdayCnt=0&currentPage=1


In [3]:
# urllib vs requests
# 1) 데이터를 보내는 방식 : urllib(바이너리), requests(딕셔너리)
#                           여러 파라미터들을 dict 형태로 지정하면 requests 편한
# 2) 없는 페이지를 요청했을 때 : urllib(에러), requests(에러는 없음)
#    FM으로 하시려면 ok인지 아닌지 code값을 통한 에러처리를 해야한다.

In [4]:
# requests 패키지로 url을 full로 요청할 때
res = requests.get(url)
res      # <Response [200]> : ok 정상적으로 반응이 온 상태

<Response [200]>

In [ ]:
# requests 패키지에서 받은 정보를 확인하는 방법이 크게 2가지 있다
# .text : 니가 알아서 인코딩/디코딩 해줘
#         --> 우리가 특별하게 인/디코딩 신경을 안 씀 ( ex)UTF-8 )
# .content : 디코딩을 바이트값으로 출력
#           --> 이미지(픽셀에 있는 값 자체가 중요한 경우, 숫자를 그대로 댕겨와야 하는 경우), 동영상 같은 경우에 주로!!
# 간략 결론) 일반적인 데이터/페이지 : text
#            이미지 : content


In [5]:
res.content

b'<!DOCTYPE html>\n\n\n\n\n\n\n\n\n\n\n\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">\n\t<head>\n\t\t<title>\xec\xb5\x9c\xea\xb7\xbc\xea\xb3\xb5\xec\x8b\x9c</title>\n\t\t<meta charset="UTF-8">\n\t\t<meta id="Viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=yes">\n\t\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\n\t\t\n\n\n\t\n<script type="text/javascript" src="/resource/js/jquery-3.3.1.min.js"></script>\n\n<link rel="stylesheet" href="/js/jquery-ui/jquery-ui.min.css">\n<script type="text/javascript" src="/js/jquery-ui/jquery-ui.min.js"></script>\n\n<!-- 2011.11.01 ext 2.3 -->\n<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->\n<script type="text/javascript" src="/js/ext-main/adapter/ext/ext-base.js"></script>\n<script type="text/javascript" src="/js/ext-main/ext-all.js"></script>\n\n<!-- x-xeries js libraries  

In [6]:
res.text

'<!DOCTYPE html>\n\n\n\n\n\n\n\n\n\n\n\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">\n\t<head>\n\t\t<title>최근공시</title>\n\t\t<meta charset="UTF-8">\n\t\t<meta id="Viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=yes">\n\t\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\n\t\t\n\n\n\t\n<script type="text/javascript" src="/resource/js/jquery-3.3.1.min.js"></script>\n\n<link rel="stylesheet" href="/js/jquery-ui/jquery-ui.min.css">\n<script type="text/javascript" src="/js/jquery-ui/jquery-ui.min.js"></script>\n\n<!-- 2011.11.01 ext 2.3 -->\n<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->\n<script type="text/javascript" src="/js/ext-main/adapter/ext/ext-base.js"></script>\n<script type="text/javascript" src="/js/ext-main/ext-all.js"></script>\n\n<!-- x-xeries js libraries  -->\n<script type="text/javascript" src="/js/

In [8]:
# 웹페이지에서 원하는 정보를 쉽게 접근하는 패키지 : bs4
soup = BeautifulSoup(res.text, "html.parser")
soup
# 이제 태그 중심으로 파일 접근이 가능해진다

<!DOCTYPE html>

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>최근공시</title>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=yes" id="Viewport" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<script src="/resource/js/jquery-3.3.1.min.js" type="text/javascript"></script>
<link href="/js/jquery-ui/jquery-ui.min.css" rel="stylesheet"/>
<script src="/js/jquery-ui/jquery-ui.min.js" type="text/javascript"></script>
<!-- 2011.11.01 ext 2.3 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->
<script src="/js/ext-main/adapter/ext/ext-base.js" type="text/javascript"></script>
<script src="/js/ext-main/ext-all.js" type="text/javascript"></script>
<!-- x-xeries js libraries  -->
<script src="/js/xjs.js?ver=1.17" type="text/javascript"></script>
<!-- application js libraries -->
<script sr

In [ ]:
# 찾을 때 tag값만 알고, 몇 번째 인지 모를 때 탐색~~~

In [10]:
len(soup.find_all("div"))

58

In [11]:
for i in range(len(soup.find_all("div"))):
    print(i)
    print(soup.find_all("div")[i])
    print("-"*30)

0
<div class="loadingMask" id="loadingMask" style="display:none"></div>
------------------------------
1
<div class="loadingWrap" id="loadingWrap" style="display:none">
<div class="loadingImg"></div>
<div class="loadingMessage">잠시만 기다려주세요.</div>
</div>
------------------------------
2
<div class="loadingImg"></div>
------------------------------
3
<div class="loadingMessage">잠시만 기다려주세요.</div>
------------------------------
4
<div class="skipMenu"><a href="#contents">본문 바로가기</a></div>
------------------------------
5
<div class="wrapper subPageBg">
<script type="text/javascript">
function openReportViewerEng(rcpNo, dcmNo){
	var url = "/dsbh001/main.do?rcpNo=" + rcpNo;
	if (dcmNo) {
		//<![CDATA[
		url += "&dcmNo=" + dcmNo;
		//]]>
	}

	var size = getOpenSize(1024, 768);
	window.open(url, rcpNo, size+",resizable=yes");
	$j("#r_"+rcpNo).css("color","#cd0093");
}

var win_num = 1;
function openReportViewer(rcpNo, dcmNo){
	var url = "/dsaf001/main.do?rcpNo=" + rcpNo;
	if (dcmNo) {
		//<![CD

In [ ]:
# 좀 더 타겟팅을 해보자!!
# tag : div, class = "pageInfo"

In [13]:
soup.find("div", class_= "pageInfo")

<div class="pageInfo">[1/9] [총 894건]</div>

In [14]:
soup.find("div", {"class":"pageInfo"})

<div class="pageInfo">[1/9] [총 894건]</div>

In [16]:
# 전체 데이터 수에 대한 정보
tot_num = soup.find("div", {"class":"pageInfo"}).text
tot_num

'[1/9] [총 894건]'

In [20]:
# 정규식을 통해서 접근1) 숫자 + 건 --> 규칙 --> 정규식
# re.findall(패턴, 찾을 소스) : 찾을 때
# re.sub(패턴, 변경, 소스) : 패턴을 찾아서 변경할 때
temp = re.findall(r"\d+건", tot_num)[0]
temp = re.sub("건", "", temp)
temp = int(temp) // 100 + 1
temp

9

In [21]:
tot_num

'[1/9] [총 894건]'

In [27]:
# 정규식을 통해서 접근2) 하나하나 찾아서 지워가는 방식
temp = re.findall(r"/\d+\]", tot_num)[0]
temp = re.sub(r"\/|\]","" , temp)
tot_num = int(temp)
tot_num

9

In [ ]:
########################### 필요한 정보 추출 ##########################

# 0. 1개 공시는 tr태그로 존재

In [28]:
# 0. 1개 공시는 tr태그로 존재
len(soup.find_all("tr"))

101

In [29]:
soup.find_all("tr")[0]

<tr>
<th scope="row"><label for="inpSample00">시간</label></th>
<th scope="row"><label for="inpSample00">공시대상회사</label></th>
<th scope="row"><label for="inpSample00">보고서명</label></th>
<th scope="row"><label for="inpSample00">제출인</label></th>
<th scope="row"><label for="inpSample00">접수일자</label></th>
<th scope="row"><label for="inpSample00">비고</label></th>
</tr>

In [30]:
len(soup.find("tbody").find_all("tr"))

100

# 1. 1번회사 : 도부마스크 회사의 모든 정보를 가지고 오자

In [31]:
soup.find("tbody").find_all("tr")[0]

<tr>
<td>
								
								
								
								18:58
							</td>
<td class="tL">
<span class="innerWrap">
<span class="tagCom_konex" style="cursor:default" title="코넥스시장">넥</span>
<a href="javascript:openCorpInfoNew('01109690', 'winCorpInfo', '/dsae001/selectPopup.ax');" title="도부마스크 기업개황 새창">
										도부마스크
									</a>
</span>
</td>
<td class="tL">
<a href="/dsaf001/main.do?rcpNo=20170328601049" id="r_20170328601049" onclick="openReportViewer('20170328601049'); return false;" title="감사보고서제출 공시뷰어 새창">감사보고서제출
									
		  							
								</a>
</td>
<td class="tL ellipsis" title="도부마스크">도부마스크</td>
<td>2017.03.28</td>
<td><span class="tagCom_konex_other" style="cursor:default" title="본 문서는 한국거래소 코넥스시장 소관임">넥</span><span class="tagCom_jung_other" style="cursor:default" title="본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람">정</span></td>
</tr>

# 2. 1번 회사의 공시 제출 시간 추출

In [39]:
soup.find("tbody").find_all("tr")[0].find("td")

<td>
								
								
								
								18:58
							</td>

In [42]:
# 파이썬 문자열의 기능 멧
soup.find("tbody").find_all("tr")[0].find("td").text

'\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t18:58\n\t\t\t\t\t\t\t'

In [41]:
# 참고) 정규식으로 공백제거
re.sub(r'\n|\t',"", soup.find("tbody").find_all("tr")[0].find("td").text)

'18:58'

# 3. 1번회사가 속한 시장을 찾아보자(코스피, 코스닥, 코넥스, 기타)
기존의 방식과 다른 부분은 우리가 원하는 정보가 테그인 속성!!!   
get 메서드...

In [44]:
soup.find("tbody").find_all("tr")[0].find_all("td")[1]

<td class="tL">
<span class="innerWrap">
<span class="tagCom_konex" style="cursor:default" title="코넥스시장">넥</span>
<a href="javascript:openCorpInfoNew('01109690', 'winCorpInfo', '/dsae001/selectPopup.ax');" title="도부마스크 기업개황 새창">
										도부마스크
									</a>
</span>
</td>

In [47]:
soup.find("tbody").find_all("tr")[0].find_all("td")[1].find_all("span")[1].get("title")

'코넥스시장'

# 4. 1번 회사의 이름 : 테그 사이에...--> 공백처리

In [48]:
# 중첩될 수 있으니 명확히 범위를 좁힌 후에 text로 값을 추출!!!!
soup.find("tbody").find_all("tr")[0].find_all("td")[1].find("a").text

'\n\t\t\t\t\t\t\t\t\t\t도부마스크\n\t\t\t\t\t\t\t\t\t'

In [49]:
soup.find("tbody").find_all("tr")[0].find_all("td")[1].find("a").text.strip()

'도부마스크'

# 5. 1번 회사의 코드값 : 속성값에 일부 --> 규칙 --> 정규식 : 숫자연속8개

In [50]:
soup.find("tbody").find_all("tr")[0].find_all("td")[1].find("a").get("href")

"javascript:openCorpInfoNew('01109690', 'winCorpInfo', '/dsae001/selectPopup.ax');"

In [51]:
temp = soup.find("tbody").find_all("tr")[0].find_all("td")[1].find("a").get("href")
temp

"javascript:openCorpInfoNew('01109690', 'winCorpInfo', '/dsae001/selectPopup.ax');"

In [53]:
re.findall(r"[0-9]{8}", temp)

['01109690']

In [54]:
re.findall(r"\d{8}", temp)

['01109690']

In [55]:
re.findall(r"[0-9]{8}", temp)[0]

'01109690'

#  6.1번 회사의 공시 이름을 출력 : 값 -> 공백?

In [56]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2]

<td class="tL">
<a href="/dsaf001/main.do?rcpNo=20170328601049" id="r_20170328601049" onclick="openReportViewer('20170328601049'); return false;" title="감사보고서제출 공시뷰어 새창">감사보고서제출
									
		  							
								</a>
</td>

In [57]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2].text

'\n감사보고서제출\n\t\t\t\t\t\t\t\t\t\n\t\t  \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n'

In [58]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2].text.strip()

'감사보고서제출'

# 7. 1번 회사의 공시의 고유값 rcpNo을 출력 : 테그의 속성값 => 규칙:숫자14개

In [59]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2]

<td class="tL">
<a href="/dsaf001/main.do?rcpNo=20170328601049" id="r_20170328601049" onclick="openReportViewer('20170328601049'); return false;" title="감사보고서제출 공시뷰어 새창">감사보고서제출
									
		  							
								</a>
</td>

In [60]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2].find("a")

<a href="/dsaf001/main.do?rcpNo=20170328601049" id="r_20170328601049" onclick="openReportViewer('20170328601049'); return false;" title="감사보고서제출 공시뷰어 새창">감사보고서제출
									
		  							
								</a>

In [61]:
soup.find("tbody").find_all("tr")[0].find_all("td")[2].find("a").get("href")

'/dsaf001/main.do?rcpNo=20170328601049'

In [62]:
temp = soup.find("tbody").find_all("tr")[0].find_all("td")[2].find("a").get("href")
temp

'/dsaf001/main.do?rcpNo=20170328601049'

In [63]:
re.findall(r"[0-9]{14}", temp)[0]

'20170328601049'

# 8. 1번 회사의 공시접수 일자 : 값

In [64]:
soup.find("tbody").find_all("tr")[0].find_all("td")[4].text

'2017.03.28'

In [66]:
#### idx = 0 회사에 대해서 정보 추출
idx = 99 # --> idx 변수를 중심으로 기술!!!
# idx번째 회사 정보를 담아두기
i_com_data = soup.find("tbody").find_all("tr")[idx]
# 제출 시간 출력
com_date = i_com_data.find("td").text.strip()
print("제출시간:",com_date)
# 회사 속한 시장 출력
com_cat = i_com_data.find_all("td")[1].find_all("span")[1].get("title")
print("속한시장:", com_cat)
# 회사 이름 출력
com_name = i_com_data.find_all("td")[1].find("a").text.strip()
print("회사이름:",com_name)
# 회사 코드 출력
com_code = i_com_data.find_all("td")[1].find("a").get("href")
com_code = re.findall(r"[0-9]{8}", com_code)[0]
print(f"회사코드:{com_code}")
# 공시 내용 출력
com_rep = i_com_data.find_all("td")[2].text.strip()
print(f"공시내용:{com_rep}")
# 공시 고유값 출력
com_repno = i_com_data.find_all("td")[2].find("a").get("href")
com_repno = re.findall(r"[0-9]{14}",com_repno)[0]
print(f"공시내용코드:{com_repno}")
# 날짜 출력
rep_date = i_com_data.find_all("td")[4].text
print(f"공시요청일자:{rep_date}")

제출시간: 17:30
속한시장: 코스닥시장
회사이름: KX
회사코드:00657987
공시내용:정기주주총회결과
공시내용코드:20170328900922
공시요청일자:2017.03.28


In [76]:
dart_df_temp = pd.DataFrame(
    {"report_time":"", 
      "com_name":"",
     "com_cat":"",
     "com_id":"",
      "rcp_name":"", 
     "rcp_id":"", 
     "report_date":""
     }, index=[9]
)

In [67]:
for i in range(100):
    #### idx = 0 회사에 대해서 정보 추출
    idx = i # --> idx 변수를 중심으로 기술!!!
    # idx번째 회사 정보를 담아두기
    i_com_data = soup.find("tbody").find_all("tr")[idx]
    # 제출 시간 출력
    com_date = i_com_data.find("td").text.strip()
    print("제출시간:",com_date)
    # 회사 속한 시장 출력
    com_cat = i_com_data.find_all("td")[1].find_all("span")[1].get("title")
    print("속한시장:", com_cat)
    # 회사 이름 출력
    com_name = i_com_data.find_all("td")[1].find("a").text.strip()
    print("회사이름:",com_name)
    # 회사 코드 출력
    com_code = i_com_data.find_all("td")[1].find("a").get("href")
    com_code = re.findall(r"[0-9]{8}", com_code)[0]
    print(f"회사코드:{com_code}")
    # 공시 내용 출력
    com_rep = i_com_data.find_all("td")[2].text.strip()
    print(f"공시내용:{com_rep}")
    # 공시 고유값 출력
    com_repno = i_com_data.find_all("td")[2].find("a").get("href")
    com_repno = re.findall(r"[0-9]{14}",com_repno)[0]
    print(f"공시내용코드:{com_repno}")
    # 날짜 출력
    rep_date = i_com_data.find_all("td")[4].text
    print(f"공시요청일자:{rep_date}")
    #
    print("-"*40)

제출시간: 18:58
속한시장: 코넥스시장
회사이름: 도부마스크
회사코드:01109690
공시내용:감사보고서제출
공시내용코드:20170328601049
공시요청일자:2017.03.28
----------------------------------------
제출시간: 18:46
속한시장: 코스닥시장
회사이름: 쏠리드
회사코드:00364403
공시내용:불성실공시법인지정예고
									
		  							(공시불이행)
공시내용코드:20170328901048
공시요청일자:2017.03.28
----------------------------------------
제출시간: 18:35
속한시장: 코스닥시장
회사이름: 코디
회사코드:00308559
공시내용:주권매매거래정지
									
		  							(불성실공시법인 지정)
공시내용코드:20170328901037
공시요청일자:2017.03.28
----------------------------------------
제출시간: 18:35
속한시장: 코스닥시장
회사이름: 코디
회사코드:00308559
공시내용:불성실공시법인지정
									
		  							(공시불이행)
공시내용코드:20170328901047
공시요청일자:2017.03.28
----------------------------------------
제출시간: 18:29
속한시장: 기타법인
회사이름: 위너지스
회사코드:00207223
공시내용:주권매매거래정지
									
		  							(불성실공시법인 지정)
공시내용코드:20170328901039
공시요청일자:2017.03.28
----------------------------------------
제출시간: 18:28
속한시장: 기타법인
회사이름: 위너지스
회사코드:00207223
공시내용:불성실공시법인지정
									
		  							(공시번복)
공시내용코드:20170328901043
공시요청일자:2017.03.28
-------------------------

제출시간: 17:34
속한시장: 코스닥시장
회사이름: 컴투스
회사코드:00476498
공시내용:주요사항보고서(자기주식처분결정)
공시내용코드:20170328000806
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:33
속한시장: 기타법인
회사이름: 이엘케이
회사코드:00379016
공시내용:주요사항보고서(유상증자결정)
공시내용코드:20170328000802
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:33
속한시장: 기타법인
회사이름: 한국인프라관리
회사코드:00842521
공시내용:감사보고서
									 (2016.12)
공시내용코드:20170328000829
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:33
속한시장: 기타법인
회사이름: 제이앤케이인더스트리
회사코드:00295370
공시내용:[기재정정]주요사항보고서(전환사채권발행결정)
공시내용코드:20170328000800
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:31
속한시장: 코스닥시장
회사이름: 동아엘텍
회사코드:00489243
공시내용:주요사항보고서(자기주식처분결정)
공시내용코드:20170328000804
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:31
속한시장: 기타법인
회사이름: 성남시민프로축구단
회사코드:00784643
공시내용:사업보고서
									 (2016.12)
공시내용코드:20170328000825
공시요청일자:2017.03.28
----------------------------------------
제출시간: 17:31
속한시장: 기타법인
회사이름: 인사사랑
회사코드:00526173
공시내용

In [78]:
num  = len(soup.find("tbody").find_all("tr"))
num

100

In [79]:
com_df = pd.DataFrame()
com_df_temp = pd.DataFrame(
    data = {
            "report_time":"", 
            "com_name":"",
            "com_cat":"",
            "com_id":"",
            "rcp_name":"", 
            "rcp_id":"", 
            "report_date":""
     }, index=[9]
)

temp_report_id = com_df_temp.columns.get_loc("report_time")
temp_comname_id = com_df_temp.columns.get_loc("com_name")
temp_comcat_id = com_df_temp.columns.get_loc("com_cat")
temp_comid_id = com_df_temp.columns.get_loc("com_id")
temp_rcpname_id = com_df_temp.columns.get_loc("rcp_name")
temp_rcpid_id = com_df_temp.columns.get_loc("rcp_id")
temp_reportdate_id = com_df_temp.columns.get_loc("report_date")

pd.concat([com_df, com_df_temp], ignore_index=True)

,report_time,com_name,com_cat,com_id,rcp_name,rcp_id,report_date
0,,,,,,,


In [ ]:
num  = len(soup.find("tbody").find_all("tr"))
for i in range(num):
    
    #### idx = 0 회사에 대해서 정보 추출
    idx = i # --> idx 변수를 중심으로 기술!!!
    # idx번째 회사 정보를 담아두기
    i_com_data = soup.find("tbody").find_all("tr")[idx]
    
    com_df_temp.iat[0, temp_report_id] = i_com_data.find("td").text.strip()   # 제출시간
    com_df_temp.iat[0, temp_comname_id] = i_com_data.find_all("td")[1].find("a").text.strip()  # 회사이름
    com_df_temp.iat[0, temp_comcat_id] = i_com_data.find_all("td")[1].find_all("span")[1].get("title") # 회사 속한 시장

    # 회사 코드 출력
    com_code = i_com_data.find_all("td")[1].find("a").get("href")
    com_code = re.findall(r"[0-9]{8}", com_code)[0]
    print(f"회사코드:{com_code}")
    # 공시 내용 출력
    com_rep = i_com_data.find_all("td")[2].text.strip()
    print(f"공시내용:{com_rep}")
    # 공시 고유값 출력
    com_repno = i_com_data.find_all("td")[2].find("a").get("href")
    com_repno = re.findall(r"[0-9]{14}",com_repno)[0]
    print(f"공시내용코드:{com_repno}")
    # 날짜 출력
    rep_date = i_com_data.find_all("td")[4].text
    print(f"공시요청일자:{rep_date}")
    #
    print("-"*40)

In [ ]:
### 1) DataFrame에서 하나씩 가져다가 쌓는 방식
### 2) dict 이용해서 통으로 변환
### 3) list, numpy : ---> deque (list의 속도적인 부분 커버)

In [84]:
data = []
table_rows = soup.find('tbody').find_all("tr")[:5]
for idx, tr in enumerate(table_rows):
    # 내가 수집할 정보
    i_com_date = ""
    i_com_name = ""
    # 실제 데이터 추출
    i_com_date = tr.find("td").text.strip()
    i_com_name = tr.find_all("td")[1].find("a").text.strip()
    # 1개 회사 정보를 다 추출되면 리스트에 쌓기
    data.append([i_com_date, i_com_name])
#     print(idx)
#     print(tr)
#     print('-------')
data

[['18:58', '도부마스크'],
 ['18:46', '쏠리드'],
 ['18:35', '코디'],
 ['18:35', '코디'],
 ['18:29', '위너지스']]

In [85]:
pd.DataFrame(data, columns=["Time", "CoName"])

,Time,CoName
0,18:58,도부마스크
1,18:46,쏠리드
2,18:35,코디
3,18:35,코디
4,18:29,위너지스


In [ ]:
data = []
# FM은 날짜만 입력으로 받고, 그에 대한 처리로 해야하는 것이니
# 그 부분은 생략이 되어 있어서 참고!!!
date = '2017.03.28'
for page in range(1, tot_num)